In [1]:
import backtrader as bt
from backtradermql5.mt5store import MTraderStore
from datetime import datetime, timedelta

%matplotlib inline

Strategy

In [2]:
# triple MA 
class MyStrategy(bt.Strategy):
    params = (
        ('ma1period', 30),
        ('ma2period', 5),
        ('ma3period', 30),
        ('atrperiod', 6),
        ('stop_loss', 0.08)
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if doprint:
            dt = dt or self.datas[0].datetime.datetime()
            print(f'{dt}: {txt}')

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.inBuyPosition = False
        self.inSellPosition = False

        #super().__init__()
        #self.broker.set_coc(True)

        # indicators
        self.ma1 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.ma1period)
        self.ma2 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.ma2period)
        self.ma3 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.ma3period)

        # entry point
        #self.crossup = bt.ind.CrossUp(self.data.close, msma, plot=True)

        # exit point
        #self.crossdown = bt.ind.CrossDown(fast_sma, slow_sma, plot=True)
        
        # entry and exit point
        #self.crossover = bt.ind.CrossOver(self.data.close, ma1, plot=True)
        #self.crossover = bt.ind.CrossOver(ma1, ma2, ma3, plot=True)

        self.atr = bt.ind.AverageTrueRange(period=self.params.atrperiod,plot=True)
        
    def notify_order(self, order):
        
        if order.status in [order.Submitted, order.Accepted]:
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.5f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                # set stop price
                #stop_price = order.executed.price * (1.0 - self.p.stop_loss)
                #self.sell(exectype=bt.Order.Stop, price=stop_price)
                
            elif order.issell():
                self.log('SELL EXECUTED, Price: %.5f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                # set stop price
                #stop_price = order.executed.price * (1.0 + self.p.stop_loss)
                #self.buy(exectype=bt.Order.Stop, price=stop_price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.5f' % self.dataclose[0])
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if
            # sma1 > sma2 > sma3
            
            if self.ma1[0] > self.ma2[0] > self.ma3[0]: 
                # BUY!!! (with all possible default parameters)
                self.log('BUY CREATE1, %.5f' % self.dataclose[0])
                self.order = self.buy()
                self.inBuyPosition = True

            if self.ma1[0] < self.ma2[0] < self.ma3[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE1, %.5f' % self.dataclose[0])
                self.order = self.sell()
                self.inSellPosition = True

        else:
            # Already in the market ..
            # we will buy if existing position is sell
            
            if self.inBuyPosition == True:
                #if self.ma1[0] < self.ma2[0] < self.ma3[0]:
                if not(self.ma1[0] > self.ma2[0] > self.ma3[0]): 
                    # SELL, SELL, SELL!!! (with all possible default parameters)
                    self.log('SELL CREATE2, %.5f' % self.dataclose[0])
                    self.order = self.sell()
                    #self.inSellPosition = True
                    self.inBuyPosition = False

                    #stop_price = self.data.close[0] * (1.0 + self.p.stop_loss)
                    #self.buy(exectype=bt.Order.Stop, price=stop_price)

            if self.inSellPosition == True:
                #if self.ma1[0] > self.ma2[0] > self.ma3[0]: 
                if not(self.ma1[0] < self.ma2[0] < self.ma3[0]):
                    # BUY!!! (with all possible default parameters)
                    self.log('BUY CREATE2, %.5f' % self.dataclose[0])
                    self.order = self.buy()
                    #self.inBuyPosition = True
                    self.inSellPosition = False
                    
                    #stop_price = self.data.close[0] * (1.0 - self.p.stop_loss)
                    #self.sell(exectype=bt.Order.Stop, price=stop_price)

    def stop(self):
        self.log('(SMA period %2d, %2d, %2d) Ending Value %.2f' %
                 (self.params.ma1period, self.params.ma2period, self.params.ma3period,
                  self.broker.getvalue()), doprint=True)
        

Main

In [12]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    #----- The Store
    store = MTraderStore(host='192.168.100.110', debug=False)

    #comment next 2 lines to use backbroker for backtesting with MTraderStore
    #broker = store.getbroker(use_positions=True)
    #cerebro.setbroker(broker)

    #start_date = datetime.now() - timedelta(minutes=720)
    start_date = datetime.now() - timedelta(days=20)
    end_date   = datetime.now() - timedelta(days=0)
    #start_date = datetime.now() - timedelta(days=360)
    #start_date=datetime(2020,3,1)
    #end_date=datetime(2019,3,1),
    #start_date = datetime(2019,1,1)
    #end_date = datetime(2020,1,1)
    
    #data = store.getdata(dataname='AUDUSD',  ## best result
    #cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal, period=17)
    #cerebro.add_signal(bt.SIGNAL_LONGEXIT, SMAExitSignal, p1=5, p2=10)
    
    #data = store.getdata(dataname='GBPUSD',
    #cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal, period=47)

    data = store.getdata(dataname='AUDUSD',
                        timeframe=bt.TimeFrame.Minutes, compression=240,
                        fromdate = start_date,
                        todate   = end_date,
                        historical=True
                        )
                        #timeframe=bt.TimeFrame.Minutes,
                        # ) #, useask=True,
                        # the parameter "useask" will request the ask price insetad if the default bid price

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    #cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=15)

    # Set our desired cash start
    cerebro.broker.setcash(10000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)
    #cerebro.addsizer(bt.sizers.FixedReverser, stake=0.5)
    #cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
    #cerebro.addsizer(bt.sizers.PercentSizerInt, percents=10)

    #----- Normal Strategy 
    #cerebro.addstrategy(SmaCross)
    #cerebro.addstrategy(St)
    #cerebro.addstrategy(TestStrategy, exitbars=5, maperiod=15) 
    #cerebro.addstrategy(MyStrategy, ma1period=5, ma2period=10, ma3period=17)

    #----- Optimizing a strategy
    #strats = cerebro.optstrategy(TestStrategy, exitbars=range(4,8)) 
    #strats = cerebro.optstrategy(MyStrategy, maperiod=range(10,41))
    strats = cerebro.optstrategy(MyStrategy, 
            ma1period=range(3,8), ma2period=range(8,13), ma3period=range(15,20))
    
    # Add signals
    #cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal, period=17)
    #cerebro.add_signal(bt.SIGNAL_LONGEXIT, SMAExitSignal, p1=5, p2=10)

    #cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal, period=47)
    
    #----- Set commission
    cerebro.broker.setcommission(commission=15,margin=1000, mult=100000)

    # test analyzer 
    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Years)
    cerebro.addanalyzer(bt.analyzers.SQN,)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    results = cerebro.run(maxcpus=1,stdstats=False)
    #results = cerebro.run()
    #cerebro.plot()
    #cerebro.plot(style='candlestick', volume=True)

    # analyzer's result
    #strat = results[0][0]
    #strat = results[0]

    # Results of own analyzers
#    al = strat.analyzers.timereturn
#    print('-- Time Return:')
#    for k, v in al.get_analysis().items():
#        print('{}: {}'.format(k, v))

#    al = strat.analyzers.sharperatio
#    print('-- Sharpe Ratio:')
#    for k, v in al.get_analysis().items():
#        print('{}: {}'.format(k, v))

#    al = strat.analyzers.sqn
#    print('-- SQN:')
#    for k, v in al.get_analysis().items():
#        print('{}: {}'.format(k, v))
        
    # Print out the final result
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2020-03-18 12:00:00: (SMA period  3,  8, 15) Ending Value 14475.00
2020-03-18 12:00:00: (SMA period  3,  8, 16) Ending Value 14341.00
2020-03-18 12:00:00: (SMA period  3,  8, 17) Ending Value 13974.00
2020-03-18 12:00:00: (SMA period  3,  8, 18) Ending Value 13904.00
2020-03-18 12:00:00: (SMA period  3,  8, 19) Ending Value 13798.00
2020-03-18 12:00:00: (SMA period  3,  9, 15) Ending Value 15038.00
2020-03-18 12:00:00: (SMA period  3,  9, 16) Ending Value 14557.00
2020-03-18 12:00:00: (SMA period  3,  9, 17) Ending Value 13609.00
2020-03-18 12:00:00: (SMA period  3,  9, 18) Ending Value 13877.00
2020-03-18 12:00:00: (SMA period  3,  9, 19) Ending Value 13771.00
2020-03-18 12:00:00: (SMA period  3, 10, 15) Ending Value 14547.00
2020-03-18 12:00:00: (SMA period  3, 10, 16) Ending Value 14434.00
2020-03-18 12:00:00: (SMA period  3, 10, 17) Ending Value 14067.00
2020-03-18 12:00:00: (SMA period  3, 10, 18) Ending Value 14335.00
2020-03-18 12:00:00: (SMA p

2020-03-18 12:00:00: (SMA period  7, 12, 18) Ending Value 14774.00
2020-03-18 12:00:00: (SMA period  7, 12, 19) Ending Value 14668.00
